In [1]:
import pandas as pd
import os
from pandas_profiling import ProfileReport

In [2]:
#Read data to DataFrame
df = pd.read_json(os.path.join(".", "data", "bgg", "bgg_info.json")).T
df.head()

,bgg_rank,name,url,year,geek_rating,avg_rating,num_voters,player_num,play_time,min_age,...,artists,publishers,categories,mechanisms,family,designers,artist,publisher,category,mechanism
10000,1,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,2017,8.576,8.83,34829,1–4 Players,60–120 Min,Age: 12+,...,"[alexandr elichev, josh t. mcdowell, alvaro ne...","[cephalofair games, albi, asmodee, feuerland s...","[adventure, exploration, fantasy, fighting, mi...","[action retrieval, campaign / battle card driv...","[campaign games, components: miniatures, crowd...",NaN,NaN,NaN,NaN,NaN
10001,2,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,2015,8.471,8.62,36928,2–4 Players,60 Min,Age: 13+,...,NaN,"[z-man games, inc., asterion press, devir, fil...","[environmental, medical]","[action points, cooperative game, hand managem...","[campaign games, legacy, pandemic]","[rob daviau, matt leacock]",[chris quilliams],NaN,NaN,NaN
10002,3,Terraforming Mars,https://boardgamegeek.com/boardgame/167791/ter...,2016,8.277,8.43,53714,1–5 Players,120 Min,Age: 12+,...,NaN,"[fryxgames, arclight, fantasmagoria, ghenos ga...","[economic, environmental, industry / manufactu...","[card drafting, end game bonuses, hand managem...","[fryxgames' future timeline, solitaire games, ...",NaN,[isaac fryxelius],NaN,NaN,NaN
10003,4,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,2018,8.269,8.63,12686,2–4 Players,60–120 Min,Age: 14+,...,"[lina cossette, david forest, damien mammoliti]","[roxley, boardm factory, conclave editora, cro...","[economic, industry / manufacturing, transport...","[connections, hand management, income, loans, ...","[brass, cities: birmingham (england), country:...","[gavan brown, matt tolman, martin wallace]",NaN,NaN,NaN,NaN
10004,5,Through the Ages: A New Story of Civilization,https://boardgamegeek.com/boardgame/182028/thr...,2015,8.221,8.47,19915,2–4 Players,120 Min,Age: 14+,...,"[filip murmak, radim pech, jakub politzer, mil...","[czech games edition, cranio creations, devir,...","[card game, civilization, economic]","[action points, auction/bidding, auction: dutc...","[tableau building, through the ages]",NaN,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117086 entries, 10000 to 11710085
Data columns (total 25 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   bgg_rank         117086 non-null  object
 1   name             117086 non-null  object
 2   url              117086 non-null  object
 3   year             107781 non-null  object
 4   geek_rating      117086 non-null  object
 5   avg_rating       117086 non-null  object
 6   num_voters       117086 non-null  object
 7   player_num       8700 non-null    object
 8   play_time        8700 non-null    object
 9   min_age          8700 non-null    object
 10  weight           8700 non-null    object
 11  primary name     7519 non-null    object
 12  alternate names  7519 non-null    object
 13  year published   7519 non-null    object
 14  designer         5207 non-null    object
 15  artists          3159 non-null    object
 16  publishers       4490 non-null    object
 17  cate

In [6]:
#Merge same columns with plural and singular form
df['categories'].where(df['categories'].notnull(), df['category'], inplace = True)
df['publishers'].where(df['publishers'].notnull(), df['publisher'], inplace = True)
df['mechanisms'].where(df['mechanisms'].notnull(), df['mechanism'], inplace = True)
df['designers'].where(df['designers'].notnull(), df['designer'], inplace = True)
df['artists'].where(df['artists'].notnull(), df['artist'], inplace = True)

#Drop boardgames with no year or mechanisms. Latter indicates missing basic info.
df =df[(df['mechanisms'].notna()) & (df['year'].notna())]

#Split player_num column to min and max player number.
df[['min_player_num', 'max_player_num']] = df['player_num'].str.split(' ', expand = True)[0].str.split('–', expand = True)
df['min_player_num'].replace({'Players': None}, inplace = True)
df['max_player_num'].where(df['max_player_num'].notnull(), df['min_player_num'], inplace = True)

#Remove non-numeric characters form weight column 
df['weight'] = df['weight'].str.split(':', expand = True)[1].str.split('/', expand = True)[0]
df['weight'].replace({' ––': None}, inplace = True)

#Remove non-numeric characters form min_age column 
df['min_age'] = df['min_age'].str.split(':', expand = True)[1].str[:-1]
df['min_age'].replace({' –': None}, inplace = True)

#Split play_time column to min and max play time.
df[['min_play_time', 'max_play_time']] = df['play_time'].str.split(' ', expand = True)[0].str.split('–', expand = True)
df['min_play_time'].replace({'Time:': None}, inplace = True)
df['max_play_time'].where(df['max_play_time'].notnull(), df['min_play_time'], inplace = True)

#Change columns datatypes 
df = df.astype({'bgg_rank': 'int', 'year': 'int', 'geek_rating': 'float', 'avg_rating': 'float',
           'num_voters': 'int', 'weight': 'float', 'min_player_num': 'float', 'max_player_num': 'float',
               'min_age': 'float', 'min_play_time': 'float', 'max_play_time': 'float', 'name': 'str'})


#Drop unnecessary columns
df.drop(['url', 'primary name', 'alternate names', 'year published', 'category', 
        'publisher', 'mechanism', 'designer', 'artist', 'play_time', 'player_num'], axis=1, inplace = True)

c:\programok\python\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
c:\programok\python\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
c:\programok\python\lib\site-packages\pandas\core\generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

In [ ]:
df.to_json(os.path.join(".", "data", "bgg", 'bgg_cleaned_data.json'))

In [ ]:
profile = ProfileReport(df, title='Pandas Profiling Report', html={'style':{'full_width':True}})
profile.to_file("report.html")

In [10]:
df = pd.read_json(os.path.join(".", "data", "bgg", "bgg_cleaned_data.json"))

In [22]:
df.groupby(['year', 'min_age']).count().tail(10)

bgg_rank  name  geek_rating  avg_rating  num_voters  weight  \
year min_age                                                                
2019 18.0            4     4            4           4           4       3   
2020 8.0             7     7            7           7           7       7   
     10.0            9     9            9           9           9       9   
     12.0            7     7            7           7           7       7   
     13.0            5     5            5           5           5       5   
     14.0           23    23           23          23          23      22   
     15.0            1     1            1           1           1       1   
     18.0            1     1            1           1           1       1   
2021 10.0            2     2            2           2           2       2   
     14.0            2     2            2           2           2       2   

              artists  publishers  categories  mechanisms  family  designers  \
year min_age                                                                   
2019 18.0           4           4           4           4       4          4   
2020 8.0            7           7           7           7       7          7   
     10.0           9           9           9           9       9          9   
     12.0           7           7           7           7       7          7   
     13.0           5           5           5           5       5          5   
     14.0          23          23          23          23      23         23   
     15.0           1           1           1           1       1          1   
     18.0           1           1           1           1       1          1   
2021 10.0           2           2           2           2       2          2   
     14.0           2           2           2           2       2          2   

              min_player_num  max_player_num  min_play_time  max_play_time  
year min_age                                                                
2019 18.0                  4               4              4              4  
2020 8.0                   7               7              7              7  
     10.0                  9               9              9              9  
     12.0                  7               7              7              7  
     13.0                  5               5              5              5  
     14.0                 23              23             23             23  
     15.0                  1               1              1              1  
     18.0                  1               1              1              1  
2021 10.0                  2               2              2              2  
     14.0                  2               2              2              2